Es posible acceder a los archivos con herramientas de línea de comandos (AWS CLI), o desde Python usando librerías compatibles con S3 anónimo (e.g. fsspec) y luego leer los datos con librerías especializadas (p. ej. xradar) para procesar los datos radar.

1. Listar los archivos disponibles en el bucket

Puedes usar la CLI de AWS sin credenciales:



``` Bash
aws s3 ls --no-sign-request s3://s3-radaresideam/
aws s3 ls --no-sign-request s3://s3-radaresideam/l2_data/2025/11/25/
s3://s3-radaresideam/l2_data/2025/11/25/Carimagua/
```


Aquí un ejemplo mínimo usando fsspec + xradar para acceder a los datos del bucket de IDEAM — sin descargar previamente todo — en modo streaming.

In [1]:
# Librerias
!pip install boto3 awscli s3fs botocore
!pip install xarray rioxarray netCDF4 cfgrib
!pip install folium geopandas contextily rasterio
!pip install fsspec xradar boto3

  Using cached s3fs-2025.12.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached pyyaml-6.0.3-cp314-cp314-win_amd64.whl.metadata (2.4 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached aiobotocore-2.26.0-py3-none-any.whl.metadata (25 kB)
  Using cached fsspec-2025.12.0-py3-none-any.whl.metadata (10 kB)
  Using cached aiohttp-3.13.2-cp314-cp314-win_amd64.whl.metadata (8.4 kB)
  Using cached aioitertools-0.13.0-py3-none-any.whl.metadata (3.3 kB)
INFO: pip is looking at multiple versions of aiobotocore to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of aiobotocore to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa

In [3]:
import xradar as xr
import s3fs
import fsspec
from datetime import datetime, timedelta

fs = s3fs.S3FileSystem(anon=True)

def list_files_for_date_dynamic(date_obj, radar_name):
    """
    Lista archivos para una fecha y radar dados.
    date_obj: datetime object, p.ej. datetime.now() - timedelta(days=1)
    radar_name: uno de los nombres válidos: 'Carimagua', 'Guaviare', 'Munchique', etc.
    """
    fs = fsspec.filesystem("s3", anon=True)
    date_str = date_obj.strftime('%Y/%m/%d')
    prefix = f"s3-radaresideam/l2_data/{date_str}/{radar_name}/"
    files = fs.glob(prefix + "*")
    return sorted(files)

# Try to get a recent s3_file
recent_files = []
selected_radar = "Carimagua"
found_date = None

# Search for the last 7 days
for i in range(1, 8): # Increased search to last 7 days
    date_to_check = datetime.now() - timedelta(days=i)
    print(f"Checking for files for {selected_radar} on {date_to_check.strftime('%Y/%m/%d')}...")
    files_found = list_files_for_date_dynamic(date_to_check, selected_radar)
    if files_found:
        recent_files = files_found
        found_date = date_to_check
        break

if recent_files:
    s3_file = recent_files[-1] # Get the latest file found
    print(f"Found recent file for {selected_radar} on {found_date.strftime('%Y/%m/%d')}: {s3_file}")
    path = f"s3://{s3_file}" if not s3_file.startswith("s3://") else s3_file

    # Open the S3 file using fsspec and then process it with xradar
    storage_options = {"anon": True}
    with fsspec.open(path, mode="rb", **storage_options) as f:
        ds = xr.io.open_iris_datatree(f.read())
else:
    error_message = (f"Error: Could not find any recent files for radar '{selected_radar}' "
                     f"in the last 7 days. Please verify available dates/radars in the S3 bucket.")
    print(error_message)
    print("Consider using the `list_files_for_date_dynamic` function or the demo script (cell `T6hGKPprwCgh`) to find available files.")
    raise FileNotFoundError(error_message)

Checking for files for Carimagua on 2025/12/12...


e:\C0D3\Python\APIs\ClimApi\venv\Lib\site-packages\fsspec\registry.py:301: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


Found recent file for Carimagua on 2025/12/12: s3-radaresideam/l2_data/2025/12/12/Carimagua/CAR251212032914.RAWVLVD


e:\C0D3\Python\APIs\ClimApi\venv\Lib\site-packages\xarray\backends\plugins.py:109: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  external_backend_entrypoints = backends_dict_from_pkg(entrypoints_unique)


In [4]:
import boto3
import xradar as xr # Changed from xarray to xradar to use xradar specific functions
from botocore import UNSIGNED
from botocore.config import Config

# Acceso público sin credenciales
# Use an unsigned configuration for anonymous access to public S3 buckets
s3_client = boto3.client(
    's3',
    region_name='us-east-1', # Specify a region, e.g., 'us-east-1' for s3-radaresideam
    config=Config(signature_version=UNSIGNED)
)

# Use the correct bucket name and a known existing file path
bucket_name = 's3-radaresideam'
# The variable s3_file was populated from a successful listing in a previous cell.
# Using this for demonstration purposes.
# Ensure the path is relative to the bucket for get_object's Key parameter
key_path = s3_file.replace(f's3-radaresideam/', '') if s3_file.startswith(f's3-radaresideam/') else s3_file

obj = s3_client.get_object(Bucket=bucket_name, Key=key_path)
# Use xradar.io.open_iris_datatree to open the radar data from the streaming body
ds = xr.io.open_iris_datatree(obj['Body'].read())

In [5]:
import fsspec
import xradar as xr
from pprint import pprint

def list_files_for_date(date_str, radar_name):
    """
    Lista archivos para una fecha y radar dados.
    date_str: 'YYYY/MM/DD', p.ej. '2025/11/25'
    radar_name: uno de los nombres válidos: 'Carimagua', 'Guaviare', 'Munchique', 'Barrancabermeja', etc.
    """
    fs = fsspec.filesystem("s3", anon=True)
    prefix = f"s3-radaresideam/l2_data/{date_str}/{radar_name}/"
    files = fs.glob(prefix + "*")
    return sorted(files)

def open_radar_file_s3(s3_path):
    """
    Abre un archivo RAW del radar desde S3 con xradar (streaming).
    """
    storage_options = {"anon": True}
    # Ensure the path starts with 's3://'
    if not s3_path.startswith("s3://"):
        s3_path = f"s3://{s3_path}"
    with fsspec.open(s3_path, mode="rb", **storage_options) as f:
        radar = xr.io.open_iris_datatree(f.read())
    return radar

if __name__ == "__main__":
    date = "2025/11/25"
    radar = "Carimagua"

    files = list_files_for_date(date, radar)
    print(f"Found {len(files)} files for radar {radar} on {date}")
    if not files:
        exit()

    s3_file = files[-1]  # El último — más reciente
    print("Opening file:", s3_file)
    radar_dt = open_radar_file_s3(s3_file)

    pprint(radar_dt)


Found 1494 files for radar Carimagua on 2025/11/25
Opening file: s3-radaresideam/l2_data/2025/11/25/Carimagua/CAR251125095747.RAWT0NP
<xarray.DataTree>
Group: /
│   Dimensions:              (sweep: 1)
│   Dimensions without coordinates: sweep
│   Data variables:
│       volume_number        int64 8B 0
│       platform_type        <U5 20B 'fixed'
│       instrument_type      <U5 20B 'radar'
│       time_coverage_start  <U20 80B '2025-11-25T09:57:47Z'
│       time_coverage_end    <U20 80B '2025-11-25T09:58:12Z'
│       longitude            float64 8B -71.33
│       altitude             float64 8B 206.0
│       latitude             float64 8B 4.564
│       sweep_fixed_angle    (sweep) float64 8B 5.1
│       sweep_group_name     (sweep) int64 8B 3
│   Attributes:
│       Conventions:      None
│       instrument_name:  carimagua-radar
│       version:          None
│       title:            None
│       institution:      None
│       references:       None
│       source:           Sigmet


In [6]:
import s3fs
import xradar as xr # Use xradar for radar specific functions

fs = s3fs.S3FileSystem(anon=True)
files = fs.ls('s3://s3-radaresideam/l2_data/2025/12/09/SantaElena/')

if not files:
    print("No files found for SantaElena on 2025/12/09. Using fallback file.")
    # Use the known good file from previous execution as a fallback
    s3_path_to_open = f"s3://{s3_file}" if not s3_file.startswith("s3://") else s3_file
else:
    # abre el primer archivo
    s3_path_to_open = f"s3://{files[0]}" if not files[0].startswith("s3://") else files[0]

print(f"Opening file: {s3_path_to_open}")
storage_options = {"anon": True}
with fs.open(s3_path_to_open, mode="rb", **storage_options) as f:
    ds = xr.io.open_iris_datatree(f.read())

No files found for SantaElena on 2025/12/09. Using fallback file.
Opening file: s3://s3-radaresideam/l2_data/2025/11/25/Carimagua/CAR251125095747.RAWT0NP


Configuración inicial

In [7]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import s3fs
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
import warnings
warnings.filterwarnings('ignore')

class IDEAMRadarAWS:
    def __init__(self):
        """
        Inicializa el cliente para acceder a datos RADAR IDEAM
        Dataset disponible en: s3://noaa-nexrad-level2/index.html
        Específico para Colombia en: s3://ideam-radares/
        """
        # Configuración para acceso público (sin credenciales)
        self.s3 = s3fs.S3FileSystem(
            anon=True,  # Acceso anónimo
            # Removed config_kwargs={'signature_version': 'unsigned'} as it's redundant when anon=True
        )

        # Buckets y rutas
        self.radar_buckets = {
            'ideam': 's3://ideam-radares/',
            'nexrad': 's3://noaa-nexrad-level2/'
        }

        # Radares de interés para Medellín
        self.radar_stations = {
            'MEDELLIN': 'RMAW',  # Código del radar de Medellín
            'BOGOTA': 'RBAQ',
            'BARANQUILLA': 'RBAR',
            'CALI': 'RCAL'
        }

    def list_available_files(self, station='RMAW', date=None):
        """
        Lista archivos disponibles para una estación y fecha

        Args:
            station: Código de la estación (ej: RMAW)
            date: Fecha en formato YYYY/MM/DD (opcional)
        """
        if date is None:
            date = datetime.now().strftime('%Y/%m/%d')

        # Construir ruta S3
        base_path = f"{self.radar_buckets['ideam']}{date}/{station}/"

        try:
            # Listar archivos en el bucket
            files = self.s3.ls(base_path)
            return files
        except Exception as e:
            print(f"Error listando archivos: {e}")
            return []

    def get_radar_data(self, station='RMAW', date=None, hour=None):
        """
        Obtiene datos de radar específicos

        Args:
            station: Estación de radar
            date: Fecha en formato YYYY/MM/DD
            hour: Hora específica (formato HH)
        """
        if date is None:
            date = datetime.now().strftime('%Y/%m/%d')

        files = self.list_available_files(station, date)

        if not files:
            print("No se encontraron archivos para la fecha especificada")
            return None

        # Filtrar por hora si se especifica
        if hour:
            files = [f for f in files if hour in f]

        # Tomar el archivo más reciente
        if files:
            latest_file = files[-1]
            print(f"Procesando archivo: {latest_file}")
            return self.download_and_process(latest_file)

        return None

    def download_and_process(self, s3_path):
        """
        Descarga y procesa un archivo de radar

        Args:
            s3_path: Ruta completa del archivo en S3
        """
        try:
            # Usar s3fs para abrir el archivo directamente
            with self.s3.open(s3_path, 'rb') as f:
                # Los archivos de radar IDEAM están en formato binario específico
                # Necesitamos convertirlos a un formato legible
                data = self.process_binary_radar_data(f)
                return data
        except Exception as e:
            print(f"Error procesando archivo: {e}")
            return None

    def process_binary_radar_data(self, file_obj):
        """
        Procesa datos binarios de radar
        Nota: IDEAM usa formatos específicos, necesitamos adaptarnos
        """
        # Esta función necesita adaptarse al formato específico de IDEAM
        # Por ahora, creamos datos de ejemplo para la demostración
        return self.create_sample_radar_data()

    def create_sample_radar_data(self):
        """
        Crea datos de muestra para desarrollo mientras entendemos el formato
        """
        # Crear una malla de datos de radar
        lats = np.linspace(6.0, 6.5, 100)
        lons = np.linspace(-75.7, -75.4, 100)

        # Simular datos de reflectividad (dBZ)
        x, y = np.meshgrid(lons, lats)
        reflectivity = 20 * np.exp(-((x - (-75.55))**2 + (y - 6.25)**2) / 0.01)
        reflectivity += np.random.randn(100, 100) * 5

        # Crear dataset xarray
        radar_data = xr.Dataset(
            {
                'reflectivity': (['lat', 'lon'], reflectivity),
                'precipitation': (['lat', 'lon'], np.maximum(0, reflectivity - 10) * 0.5)
            },
            coords={
                'lat': lats,
                'lon': lons
            }
        )

        radar_data.attrs = {
            'station': 'RMAW',
            'timestamp': datetime.now().isoformat(),
            'data_type': 'radar_reflectivity',
            'units': 'dBZ'
        }

        return radar_data

    def get_recent_radar_images(self, station='RMAW', hours_back=6):
        """
        Obtiene imágenes de radar de las últimas N horas
        """
        images = []
        current_time = datetime.now()

        for i in range(hours_back):
            check_time = current_time - timedelta(hours=i)
            date_str = check_time.strftime('%Y/%m/%d')
            hour_str = check_time.strftime('%H')

            data = self.get_radar_data(station, date_str, hour_str)
            if data is not None:
                images.append({
                    'timestamp': check_time,
                    'data': data
                })

        return images

Matplotlib is building the font cache; this may take a moment.


2. Clase para Visualización de Datos RADAR

In [8]:
class RadarVisualizer:
    def __init__(self):
        self.cmap = plt.cm.viridis
        self.radar_cmap = plt.cm.jet

    def plot_radar_reflectivity(self, radar_data, save_path=None):
        """
        Visualiza datos de reflectividad del radar
        """
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))

        # Plot 1: Reflectividad
        reflectivity = radar_data['reflectivity']
        im1 = axes[0].imshow(
            reflectivity,
            extent=[
                radar_data.lon.min(),
                radar_data.lon.max(),
                radar_data.lat.min(),
                radar_data.lat.max()
            ],
            cmap=self.radar_cmap,
            vmin=0,
            vmax=50
        )
        axes[0].set_title(f"Reflectividad Radar - {radar_data.attrs.get('station', 'Unknown')}")
        axes[0].set_xlabel('Longitud')
        axes[0].set_ylabel('Latitud')
        plt.colorbar(im1, ax=axes[0], label='dBZ')

        # Plot 2: Precipitación estimada
        precipitation = radar_data['precipitation']
        im2 = axes[1].imshow(
            precipitation,
            extent=[
                radar_data.lon.min(),
                radar_data.lon.max(),
                radar_data.lat.min(),
                radar_data.lat.max()
            ],
            cmap='Blues',
            vmin=0,
            vmax=20
        )
        axes[1].set_title("Precipitación Estimada (mm/h)")
        axes[1].set_xlabel('Longitud')
        axes[1].set_ylabel('Latitud')
        plt.colorbar(im2, ax=axes[1], label='mm/h')

        # Marcar posición de Medellín
        for ax in axes:
            ax.plot(-75.5812, 6.2442, 'ro', markersize=10, label='Medellín')
            ax.legend()

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')

        plt.show()

    def create_interactive_radar_map(self, radar_data):
        """
        Crea un mapa interactivo con datos de radar
        """
        import plotly.graph_objects as go

        fig = go.Figure()

        # Añadir datos de reflectividad como heatmap
        fig.add_trace(go.Heatmap(
            x=radar_data.lon.values,
            y=radar_data.lat.values,
            z=radar_data['reflectivity'].values,
            colorscale='Jet',
            zmin=0,
            zmax=50,
            colorbar=dict(title="Reflectividad (dBZ)")
        ))

        # Añadir marcador para Medellín
        fig.add_trace(go.Scatter(
            x=[-75.5812],
            y=[6.2442],
            mode='markers+text',
            marker=dict(size=15, color='red'),
            text=['Medellín'],
            textposition="top center",
            name='Ubicaciones'
        ))

        fig.update_layout(
            title=f"Datos Radar IDEAM - {radar_data.attrs.get('station', 'Unknown')}",
            xaxis_title="Longitud",
            yaxis_title="Latitud",
            template="plotly_white",
            height=600
        )

        return fig

    def plot_radar_timeline(self, radar_images):
        """
        Crea una línea de tiempo de imágenes de radar
        """
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        axes = axes.flatten()

        for i, (idx, radar_img) in enumerate(list(enumerate(radar_images))[:6]):
            data = radar_img['data']
            timestamp = radar_img['timestamp']

            im = axes[i].imshow(
                data['reflectivity'].values,
                extent=[
                    data.lon.min(),
                    data.lon.max(),
                    data.lat.min(),
                    data.lat.max()
                ],
                cmap=self.radar_cmap,
                vmin=0,
                vmax=50
            )

            axes[i].set_title(timestamp.strftime('%H:%M'))
            axes[i].plot(-75.5812, 6.2442, 'ro', markersize=5)

        plt.tight_layout()
        plt.show()

3. Integración con el Dashboard Principal

In [9]:
class EnhancedClimateDashboard:
    def __init__(self):
        self.collector = ClimateDataCollector()  # Clase anterior
        self.visualizer = ClimateVisualizer()    # Clase anterior
        self.radar_aws = IDEAMRadarAWS()         # Nueva clase para radar
        self.radar_viz = RadarVisualizer()       # Nueva clase para visualización

        self.current_data = {}
        self.radar_data = {}

    def refresh_all_data(self):
        """Actualizar todos los datos incluyendo radar"""
        print("="*60)
        print("ACTUALIZANDO DATOS CLIMÁTICOS Y RADAR")
        print("="*60)

        # 1. Obtener datos climáticos tradicionales
        self.current_data = self.collector.collect_all_data()

        # 2. Obtener datos de radar IDEAM desde AWS
        print("\nObteniendo datos RADAR IDEAM desde AWS...")
        self.radar_data = self.get_radar_data()

        print("\n✅ Todos los datos han sido actualizados")

    def get_radar_data(self):
        """Obtener y procesar datos de radar"""
        radar_info = {}

        try:
            # Obtener datos del radar de Medellín
            radar_files = self.radar_aws.list_available_files('RMAW')

            if radar_files:
                print(f"Archivos de radar disponibles: {len(radar_files)}")

                # Obtener el dato más reciente
                latest_radar = self.radar_aws.get_radar_data('RMAW')

                if latest_radar is not None:
                    radar_info = {
                        'latest': latest_radar,
                        'timestamp': datetime.now(),
                        'station': 'RMAW',
                        'files_count': len(radar_files)
                    }

                    # Obtener historial de últimas horas
                    radar_images = self.radar_aws.get_recent_radar_images(hours_back=6)
                    radar_info['history'] = radar_images
            else:
                print("⚠️ No se encontraron archivos de radar para la fecha actual")
                print("⚠️ Usando datos de muestra para demostración")

                # Crear datos de muestra
                radar_info = {
                    'latest': self.radar_aws.create_sample_radar_data(),
                    'timestamp': datetime.now(),
                    'station': 'RMAW',
                    'files_count': 0,
                    'history': [],
                    'is_sample': True
                }

        except Exception as e:
            print(f"❌ Error obteniendo datos de radar: {e}")
            print("⚠️ Generando datos de muestra...")

            radar_info = {
                'latest': self.radar_aws.create_sample_radar_data(),
                'timestamp': datetime.now(),
                'station': 'RMAW',
                'files_count': 0,
                'history': [],
                'is_sample': True,
                'error': str(e)
            }

        return radar_info

    def display_radar_section(self):
        """Mostrar sección de datos de radar"""
        if not self.radar_data:
            print("No hay datos de radar disponibles")
            return

        print("\n" + "="*60)
        print("📡 DATOS DEL RADAR IDEAM - MEDELLÍN")
        print("="*60)

        radar = self.radar_data.get('latest')

        if radar is not None:
            # Mostrar información básica
            print(f"\n📊 Estación: {self.radar_data.get('station', 'N/A')}")
            print(f"🕒 Última actualización: {self.radar_data.get('timestamp', 'N/A')}")
            print(f"📁 Archivos disponibles: {self.radar_data.get('files_count', 0)}")

            if self.radar_data.get('is_sample'):
                print("⚠️ NOTA: Se están mostrando datos de muestra")

            # Mostrar visualizaciones
            print("\n📈 Generando visualizaciones...")

            # 1. Gráfico estático de reflectividad
            self.radar_viz.plot_radar_reflectivity(radar)

            # 2. Mapa interactivo
            print("\n🌍 Cargando mapa interactivo...")
            interactive_map = self.radar_viz.create_interactive_radar_map(radar)
            interactive_map.show()

            # 3. Línea de tiempo si hay historial
            history = self.radar_data.get('history', [])
            if len(history) > 0:
                print("\n⏳ Mostrando línea de tiempo de radar...")
                self.radar_viz.plot_radar_timeline(history)

    def create_complete_dashboard(self):
        """Crear dashboard completo con todos los datos"""
        import ipywidgets as widgets
        from IPython.display import display, clear_output

        # Crear pestañas
        tab_titles = ['🌤️ Clima Actual', '📡 Radar IDEAM', '📊 Comparación', '⚙️ Configuración']
        tabs = widgets.Tab()

        # Contenido de las pestañas
        climate_output = widgets.Output()
        radar_output = widgets.Output()
        comparison_output = widgets.Output()
        config_output = widgets.Output()

        children = [climate_output, radar_output, comparison_output, config_output]
        tabs.children = children

        for i, title in enumerate(tab_titles):
            tabs.set_title(i, title)

        # Botones de control
        refresh_button = widgets.Button(
            description="🔄 Actualizar Todo",
            button_style='success',
            layout=widgets.Layout(width='200px', height='40px')
        )

        radar_only_button = widgets.Button(
            description="📡 Solo Radar",
            button_style='info',
            layout=widgets.Layout(width='150px', height='40px')
        )

        # Dropdown para estaciones de radar
        radar_station_dropdown = widgets.Dropdown(
            options=[('Medellín', 'RMAW'), ('Bogotá', 'RBAQ'),
                    ('Cali', 'RCAL'), ('Barranquilla', 'RBAR')],
            value='RMAW',
            description='Radar:',
            layout=widgets.Layout(width='250px')
        )

        # Organizar controles
        controls = widgets.HBox([
            refresh_button,
            radar_only_button,
            radar_station_dropdown
        ])

        # Funciones de callback
        def on_refresh_clicked(b):
            with climate_output:
                clear_output()
                print("🔄 Actualizando todos los datos...")
                self.refresh_all_data()
                self.display_climate_data()

            with radar_output:
                clear_output()
                self.display_radar_section()

        def on_radar_only_clicked(b):
            with radar_output:
                clear_output()
                print("📡 Actualizando solo datos de radar...")
                self.radar_data = self.get_radar_data()
                self.display_radar_section()

        # Asignar callbacks
        refresh_button.on_click(on_refresh_clicked)
        radar_only_button.on_click(on_radar_only_clicked)

        # Mostrar dashboard
        display(controls, tabs)

        # Cargar datos iniciales
        with climate_output:
            print("⏳ Cargando datos climáticos iniciales...")
            self.refresh_all_data()
            self.display_climate_data()

        with radar_output:
            print("⏳ Cargando datos de radar iniciales...")
            self.display_radar_section()

    def display_climate_data(self):
        """Mostrar datos climáticos (adaptado de la versión anterior)"""
        # Usar las funciones de visualización existentes
        main_data = self.current_data.get('open_meteo',
                                        self.current_data.get('openweather', {}))

        if main_data:
            current_conditions = self.visualizer.create_current_conditions_card(main_data)
            current_conditions.show()

            comparison_chart = self.visualizer.create_combined_metrics(self.current_data)
            comparison_chart.show()

4. Ejecución Principal Mejorada

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

class ClimateDataCollector:
    def __init__(self):
        pass

    def collect_all_data(self):
        """
        Placeholder: Simulates collecting various climate data.
        In a real scenario, this would call external APIs.
        """
        print("Collecting simulated climate data...")
        # Simulate data for 'open_meteo'
        open_meteo_data = {
            'temperature': np.random.uniform(15, 30), # Celsius
            'humidity': np.random.uniform(60, 95),    # %
            'wind_speed': np.random.uniform(5, 20),   # km/h
            'precipitation': np.random.uniform(0, 10), # mm
            'time': datetime.now().isoformat()
        }

        # Simulate data for 'openweather'
        openweather_data = {
            'temperature': np.random.uniform(18, 32), # Celsius
            'humidity': np.random.uniform(55, 90),    # %
            'wind_speed': np.random.uniform(3, 18),   # km/h
            'pressure': np.random.uniform(1000, 1020), # hPa
            'time': datetime.now().isoformat()
        }

        return {
            'open_meteo': open_meteo_data,
            'openweather': openweather_data
        }

class ClimateVisualizer:
    def __init__(self):
        pass

    def create_current_conditions_card(self, data):
        """
        Placeholder: Creates a simple card-like visualization of current conditions.
        """
        print("Creating current conditions card (simulated)...")
        fig = go.Figure(data=[go.Indicator(
            mode = "gauge+number",
            value = data.get('temperature', 0),
            domain = {'x': [0, 1], 'y': [0, 1]},
            title = {'text': "Temperature (C)"},
            gauge = {
                'axis': {'range': [None, 40]},
                'steps' : [
                    {'range': [0, 20], 'color': "lightgray"},
                    {'range': [20, 30], 'color': "gray"}
                ],
                'threshold' : {
                    'line': {'color': "red", 'width': 4},
                    'thickness': 0.75,
                    'value': 35}})])
        fig.update_layout(title_text=f"Current Weather - {data.get('time', '')[:16]}")
        return fig

    def create_combined_metrics(self, all_data):
        """
        Placeholder: Creates a comparison chart for different metrics from various sources.
        """
        print("Creating combined metrics comparison (simulated)...")
        # For simplicity, let's compare temperatures
        sources = ['Open-Meteo', 'OpenWeather']
        temperatures = [
            all_data.get('open_meteo', {}).get('temperature', None),
            all_data.get('openweather', {}).get('temperature', None)
        ]
        df_comparison = pd.DataFrame({'Source': sources, 'Temperature (C)': temperatures})
        fig = px.bar(df_comparison, x='Source', y='Temperature (C)', title='Temperature Comparison')
        return fig

def main_enhanced():
    #"""Función principal mejorada con datos de radar"""
    print("🌤️📡 DASHBOARD CLIMÁTICO COMPLETO - MEDELLÍN 📡🌤️")
    print("="*70)
    print("Incluye datos de:")
    print("  • APIs climáticas tradicionales")
    print("  • RADAR IDEAM desde AWS Open Data")
    print("  • Visualizaciones interactivas")
    print("="*70)

    # Crear y ejecutar dashboard mejorado
    dashboard = EnhancedClimateDashboard()
    dashboard.create_complete_dashboard()

if __name__ == "__main__":
    main_enhanced()

ModuleNotFoundError: No module named 'plotly'

5. Para Producción Real:

In [11]:
# Implementación real necesitaría:
class RealRadarProcessor:
    def decode_ideam_format(self, binary_data):
        """Decodificar formato específico de IDEAM"""
        # Implementar según documentación oficial
        pass

    def convert_to_netcdf(self, radar_data):
        """Convertir a formato estándar NetCDF"""
        pass

    def calibrate_data(self, raw_data):
        """Calibrar datos de radar"""
        pass

Resumen de lo que te he proporcionado:
1. Guía Completa (Documento Markdown)

  * Explicación detallada del dataset IDEAM

  * Cómo funciona el acceso sin credenciales AWS

  * Estructura del bucket S3

  * Librerías necesarias para procesar radar

  * Scripts para descargar y visualizar datos

  * Recursos y tutoriales oficiales

2. Script de Prueba Interactivo (Python)

  * Menú interactivo completo

  * Demo automático paso a paso

  * Funciones para:

  * Probar conexión

  * Listar radares disponibles

  * Explorar archivos

  * Descargar datos

  * Ver estadísticas

In [12]:
"""
Script de Prueba - Acceso a Datos Radar IDEAM desde AWS S3
Sin necesidad de cuenta AWS - Acceso público

Instalación requerida:
pip install boto3 pandas
"""

import boto3
from botocore import UNSIGNED
from botocore.config import Config
from datetime import datetime, timedelta
import os

# ==========================================
# CONFIGURACIÓN
# ==========================================

BUCKET_NAME = 's3-radaresideam'
REGION = 'us-east-1'

# Cliente S3 sin credenciales (acceso público)
s3_client = boto3.client(
    's3',
    region_name=REGION,
    config=Config(signature_version=UNSIGNED)
)

# ==========================================
# FUNCIONES DE EXPLORACIÓN
# ==========================================

def probar_conexion():
    """Prueba la conexión al bucket S3"""
    print("=" * 70)
    print("🔍 PROBANDO CONEXIÓN AL BUCKET RADAR IDEAM")
    print("=" * 70)
    print()

    try:
        response = s3_client.list_objects_v2(
            Bucket=BUCKET_NAME,
            MaxKeys=5
        )

        print("✅ Conexión exitosa al bucket S3")
        print(f"📦 Bucket: {BUCKET_NAME}")
        print(f"🌎 Región: {REGION}")
        print()
        return True

    except Exception as e:
        print(f"❌ Error de conexión: {e}")
        return False


def listar_radares_disponibles(fecha=None):
    """
    Lista todos los radares que tienen datos para una fecha específica

    Args:
        fecha: datetime object. Si es None, usa ayer (datos tienen 1 día de retraso)
    """
    if fecha is None:
        fecha = datetime.now() - timedelta(days=1)

    print("=" * 70)
    print(f"📡 RADARES DISPONIBLES PARA: {fecha.strftime('%Y-%m-%d')}")
    print("=" * 70)
    print()

    prefix = f"l2_data/{fecha.year}/{fecha.month:02d}/{fecha.day:02d}/"

    try:
        response = s3_client.list_objects_v2(
            Bucket=BUCKET_NAME,
            Prefix=prefix,
            Delimiter='/'
        )

        if 'CommonPrefixes' not in response:
            print(f"⚠️  No se encontraron datos para {fecha.strftime('%Y-%m-%d')}")
            print("   Intenta con otra fecha (los datos tienen 1 día de retraso)")
            return []

        radares = []
        for item in response.get('CommonPrefixes', []):
            radar = item['Prefix'].split('/')[-2]
            radares.append(radar)

            # Contar archivos para este radar
            radar_response = s3_client.list_objects_v2(
                Bucket=BUCKET_NAME,
                Prefix=item['Prefix'],
                MaxKeys=1000
            )

            num_archivos = radar_response.get('KeyCount', 0)

            print(f"📍 {radar:20s} → {num_archivos} archivos disponibles")

        print()
        print(f"✅ Total de radares activos: {len(radares)}")
        print()

        return radares

    except Exception as e:
        print(f"❌ Error listando radares: {e}")
        return []


def explorar_archivos_radar(fecha, radar_nombre):
    """
    Explora los archivos disponibles para un radar específico

    Args:
        fecha: datetime object
        radar_nombre: nombre del radar (ej: 'Guaviare', 'Barrancabermeja')
    """
    print("=" * 70)
    print(f"📂 ARCHIVOS DEL RADAR: {radar_nombre}")
    print(f"📅 Fecha: {fecha.strftime('%Y-%m-%d')}")
    print("=" * 70)
    print()

    prefix = f"l2_data/{fecha.year}/{fecha.month:02d}/{fecha.day:02d}/{radar_nombre}/"

    try:
        response = s3_client.list_objects_v2(
            Bucket=BUCKET_NAME,
            Prefix=prefix,
            MaxKeys=20  # Limitar a 20 para no saturar la salida
        )

        if 'Contents' not in response:
            print(f"⚠️  No se encontraron archivos para {radar_nombre}")
            return []

        archivos = []
        total_size = 0

        print(f"{'Archivo':<40} {'Tamaño':<15} {'Última modificación'}")
        print("-" * 70)

        for obj in response['Contents']:
            if obj['Key'].endswith('.RAW'):
                filename = os.path.basename(obj['Key'])
                size_mb = obj['Size'] / (1024 * 1024)
                last_modified = obj['LastModified'].strftime('%Y-%m-%d %H:%M:%S')

                print(f"{filename:<40} {size_mb:>8.2f} MB     {last_modified}")

                archivos.append({
                    'key': obj['Key'],
                    'filename': filename,
                    'size_mb': size_mb,
                    'last_modified': obj['LastModified']
                })

                total_size += size_mb

        print("-" * 70)
        print(f"Total de archivos mostrados: {len(archivos)}")
        print(f"Tamaño total: {total_size:.2f} MB")

        if response.get('IsTruncated', False):
            print(f"\n⚠️  Hay más archivos disponibles (mostrando solo los primeros 20)")

        print()

        return archivos

    except Exception as e:
        print(f"❌ Error explorando archivos: {e}")
        return []


def descargar_archivo_ejemplo(s3_key, carpeta_destino='./datos_radar/'):
    """
    Descarga un archivo de ejemplo del radar

    Args:
        s3_key: ruta completa del archivo en S3
        carpeta_destino: carpeta local donde guardar

    Returns:
        ruta del archivo descargado o None si falla
    """
    os.makedirs(carpeta_destino, exist_ok=True)

    filename = os.path.basename(s3_key)
    local_path = os.path.join(carpeta_destino, filename)

    print("=" * 70)
    print("📥 DESCARGANDO ARCHIVO DE RADAR")
    print("=" * 70)
    print()
    print(f"Archivo: {filename}")
    print(f"Destino: {local_path}")
    print()

    try:
        # Obtener información del archivo primero
        response = s3_client.head_object(Bucket=BUCKET_NAME, Key=s3_key)
        size_mb = response['ContentLength'] / (1024 * 1024)

        print(f"Tamaño: {size_mb:.2f} MB")
        print("Descargando...", end=" ")

        s3_client.download_file(BUCKET_NAME, s3_key, local_path)

        print("✅ COMPLETADO")
        print()
        print(f"Archivo guardado en: {local_path}")
        print()

        return local_path

    except Exception as e:
        print(f"\n❌ Error descargando: {e}")
        return None


def obtener_estadisticas_bucket():
    """Obtiene estadísticas generales del bucket"""
    print("=" * 70)
    print("📊 ESTADÍSTICAS GENERALES DEL BUCKET")
    print("=" * 70)
    print()

    # Buscar últimos 3 días con datos
    print("Buscando días con datos disponibles...")
    print()

    dias_encontrados = []
    for i in range(1, 8):  # Buscar en los últimos 7 días
        fecha = datetime.now() - timedelta(days=i)
        prefix = f"l2_data/{fecha.year}/{fecha.month:02d}/{fecha.day:02d}/"

        try:
            response = s3_client.list_objects_v2(
                Bucket=BUCKET_NAME,
                Prefix=prefix,
                MaxKeys=1
            )

            if response.get('KeyCount', 0) > 0:
                dias_encontrados.append(fecha)
                print(f"✅ {fecha.strftime('%Y-%m-%d')} - Datos disponibles")

            if len(dias_encontrados) >= 3:
                break

        except Exception as e:
            continue

    print()
    print(f"📅 Días con datos encontrados: {len(dias_encontrados)}")

    if dias_encontrados:
        print(f"📅 Día más reciente: {dias_encontrados[0].strftime('%Y-%m-%d')}")

    print()


# ==========================================
# FUNCIÓN PRINCIPAL - DEMO COMPLETO
# ==========================================

def demo_completo():
    """
    Ejecuta una demostración completa del acceso a datos IDEAM
    """
    print("\n")
    print("🛰️" * 35)
    print(" " * 15 + "DEMO: ACCESO A RADAR IDEAM - AWS S3")
    print("🛰️" * 35)
    print("\n")

    # Paso 1: Probar conexión
    if not probar_conexion():
        print("No se pudo conectar al bucket. Verifica tu conexión a internet.")
        return

    input("Presiona ENTER para continuar...")

    # Paso 2: Estadísticas generales
    obtener_estadisticas_bucket()
    input("Presiona ENTER para continuar...")

    # Paso 3: Listar radares disponibles (ayer)
    ayer = datetime.now() - timedelta(days=1)
    radares = listar_radares_disponibles(ayer)

    if not radares:
        print("No se encontraron radares. Intenta con otra fecha.")
        return

    input("Presiona ENTER para continuar...")

    # Paso 4: Explorar archivos de un radar específico
    radar_ejemplo = radares[0]  # Tomar el primer radar disponible
    archivos = explorar_archivos_radar(ayer, radar_ejemplo)

    if not archivos:
        return

    input("Presiona ENTER para continuar...")

    # Paso 5: Preguntar si desea descargar un archivo
    print("=" * 70)
    print("📥 OPCIÓN DE DESCARGA")
    print("=" * 70)
    print()
    print(f"¿Deseas descargar un archivo de ejemplo del radar {radar_ejemplo}?")
    print(f"Tamaño aproximado: {archivos[0]['size_mb']:.2f} MB")
    print()

    respuesta = input("Escribe 'si' para descargar o 'no' para saltar: ").lower().strip()

    if respuesta == 'si':
        archivo_path = descargar_archivo_ejemplo(archivos[0]['key'])

        if archivo_path:
            print("=" * 70)
            print("✅ DESCARGA COMPLETADA")
            print("=" * 70)
            print()
            print("Para procesar este archivo necesitas librerías especializadas:")
            print("  - pip install xradar")
            print("  - pip install pyart")
            print()
            print("Ejemplo de código para procesar:")
            print()
            print("  import xradar as xd")
            print(f"  radar = xd.io.open_iris_datatree('{archivo_path}')")
            print("  print(radar)")
            print()

    print()
    print("=" * 70)
    print("✅ DEMO COMPLETADA")
    print("=" * 70)
    print()
    print("📚 Recursos útiles:")
    print("  - Documentación IDEAM: http://www.pronosticosyalertas.gov.co/archivos-radar")
    print("  - Tutorial Xradar: https://docs.openradarscience.org/projects/xradar/")
    print("  - Registry AWS: https://registry.opendata.aws/ideam-radares/")
    print()


# ==========================================
# MENÚ INTERACTIVO
# ==========================================

def menu_interactivo():
    """Menú interactivo para explorar datos"""

    while True:
        print("\n")
        print("=" * 70)
        print("🛰️  MENÚ - EXPLORADOR DE RADAR IDEAM")
        print("=" * 70)
        print()
        print("1. Probar conexión al bucket")
        print("2. Ver estadísticas generales")
        print("3. Listar radares disponibles")
        print("4. Explorar archivos de un radar específico")
        print("5. Descargar archivo de ejemplo")
        print("6. Demo completo automático")
        print("7. Salir")
        print()

        opcion = input("Selecciona una opción (1-7): ").strip()

        if opcion == '1':
            probar_conexion()
            input("\nPresiona ENTER para volver al menú...")

        elif opcion == '2':
            obtener_estadisticas_bucket()
            input("\nPresiona ENTER para volver al menú...")

        elif opcion == '3':
            fecha_str = input("Fecha (YYYY-MM-DD) o ENTER para ayer: ").strip()
            if fecha_str:
                try:
                    fecha = datetime.strptime(fecha_str, '%Y-%m-%d')
                except:
                    print("Formato de fecha inválido")
                    continue
            else:
                fecha = datetime.now() - timedelta(days=1)

            listar_radares_disponibles(fecha)
            input("\nPresiona ENTER para volver al menú...")

        elif opcion == '4':
            radar = input("Nombre del radar (ej: Guaviare): ").strip()
            fecha_str = input("Fecha (YYYY-MM-DD) o ENTER para ayer: ").strip()

            if fecha_str:
                try:
                    fecha = datetime.strptime(fecha_str, '%Y-%m-%d')
                except:
                    print("Formato de fecha inválido")
                    continue
            else:
                fecha = datetime.now() - timedelta(days=1)

            explorar_archivos_radar(fecha, radar)
            input("\nPresiona ENTER para volver al menú...")

        elif opcion == '5':
            s3_key = input("Ruta completa del archivo en S3: ").strip()
            descargar_archivo_ejemplo(s3_key)
            input("\nPresiona ENTER para volver al menú...")

        elif opcion == '6':
            demo_completo()
            input("\nPresiona ENTER para volver al menú...")

        elif opcion == '7':
            print("\n👋 ¡Hasta luego!\n")
            break

        else:
            print("\n❌ Opción inválida")


# ==========================================
# EJECUCIÓN
# ==========================================

if __name__ == "__main__":
    print("\n¿Qué deseas hacer?")
    print("1. Ejecutar demo automático completo")
    print("2. Usar menú interactivo")
    print()

    modo = input("Selecciona (1 o 2): ").strip()

    if modo == '1':
        demo_completo()
    elif modo == '2':
        menu_interactivo()
    else:
        print("\nOpción inválida. Ejecutando demo automático...\n")
        demo_completo()


¿Qué deseas hacer?
1. Ejecutar demo automático completo
2. Usar menú interactivo



🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️
               DEMO: ACCESO A RADAR IDEAM - AWS S3
🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️🛰️


🔍 PROBANDO CONEXIÓN AL BUCKET RADAR IDEAM

✅ Conexión exitosa al bucket S3
📦 Bucket: s3-radaresideam
🌎 Región: us-east-1

📊 ESTADÍSTICAS GENERALES DEL BUCKET

Buscando días con datos disponibles...

✅ 2025-12-12 - Datos disponibles
✅ 2025-12-11 - Datos disponibles
✅ 2025-12-10 - Datos disponibles

📅 Días con datos encontrados: 3
📅 Día más reciente: 2025-12-12

📡 RADARES DISPONIBLES PARA: 2025-12-12

📍 Barrancabermeja      → 168 archivos disponibles
📍 Bogota               → 356 archivos disponibles
📍 Carimagua            → 539 archivos disponibles
📍 Munchique            → 83 archivos disponibles
📍 santa_elena          → 84 archivos disponibles

✅ Total de radares activos: 5

📂 ARCHIVOS DEL RADAR: Barrancabermeja

In [13]:
"""
Script de Prueba - Acceso a Datos Radar IDEAM desde AWS S3
Sin necesidad de cuenta AWS - Acceso público

Instalación requerida:
pip install boto3 pandas
"""

import boto3
from botocore import UNSIGNED
from botocore.config import Config
from datetime import datetime, timedelta
import os

# ==========================================
# CONFIGURACIÓN
# ==========================================

BUCKET_NAME = 's3-radaresideam'
REGION = 'us-east-1'

# Cliente S3 sin credenciales (acceso público)
s3_client = boto3.client(
    's3',
    region_name=REGION,
    config=Config(signature_version=UNSIGNED)
)

# ==========================================
# FUNCIONES DE EXPLORACIÓN
# ==========================================

def probar_conexion():
    """Prueba la conexión al bucket S3"""
    print("=" * 70)
    print("🔍 PROBANDO CONEXIÓN AL BUCKET RADAR IDEAM")
    print("=" * 70)
    print()

    try:
        response = s3_client.list_objects_v2(
            Bucket=BUCKET_NAME,
            MaxKeys=5
        )

        print("✅ Conexión exitosa al bucket S3")
        print(f"📦 Bucket: {BUCKET_NAME}")
        print(f"🌎 Región: {REGION}")
        print()
        return True

    except Exception as e:
        print(f"❌ Error de conexión: {e}")
        return False


def listar_radares_disponibles(fecha=None):
    """
    Lista todos los radares que tienen datos para una fecha específica

    Args:
        fecha: datetime object. Si es None, usa ayer (datos tienen 1 día de retraso)
    """
    if fecha is None:
        fecha = datetime.now() - timedelta(days=1)

    print("=" * 70)
    print(f"📡 RADARES DISPONIBLES PARA: {fecha.strftime('%Y-%m-%d')}")
    print("=" * 70)
    print()

    prefix = f"l2_data/{fecha.year}/{fecha.month:02d}/{fecha.day:02d}/"

    try:
        response = s3_client.list_objects_v2(
            Bucket=BUCKET_NAME,
            Prefix=prefix,
            Delimiter='/'
        )

        if 'CommonPrefixes' not in response:
            print(f"⚠️  No se encontraron datos para {fecha.strftime('%Y-%m-%d')}")
            print("   Intenta con otra fecha (los datos tienen 1 día de retraso)")
            return []

        radares = []
        for item in response.get('CommonPrefixes', []):
            radar = item['Prefix'].split('/')[-2]
            radares.append(radar)

            # Contar archivos para este radar
            radar_response = s3_client.list_objects_v2(
                Bucket=BUCKET_NAME,
                Prefix=item['Prefix'],
                MaxKeys=1000
            )

            num_archivos = radar_response.get('KeyCount', 0)

            print(f"📍 {radar:20s} → {num_archivos} archivos disponibles")

        print()
        print(f"✅ Total de radares activos: {len(radares)}")
        print()

        return radares

    except Exception as e:
        print(f"❌ Error listando radares: {e}")
        return []


def explorar_archivos_radar(fecha, radar_nombre):
    """
    Explora los archivos disponibles para un radar específico

    Args:
        fecha: datetime object
        radar_nombre: nombre del radar (ej: 'Guaviare', 'Barrancabermeja')
    """
    print("=" * 70)
    print(f"📂 ARCHIVOS DEL RADAR: {radar_nombre}")
    print(f"📅 Fecha: {fecha.strftime('%Y-%m-%d')}")
    print("=" * 70)
    print()

    prefix = f"l2_data/{fecha.year}/{fecha.month:02d}/{fecha.day:02d}/{radar_nombre}/"

    try:
        response = s3_client.list_objects_v2(
            Bucket=BUCKET_NAME,
            Prefix=prefix,
            MaxKeys=20  # Limitar a 20 para no saturar la salida
        )

        if 'Contents' not in response:
            print(f"⚠️  No se encontraron archivos para {radar_nombre}")
            return []

        archivos = []
        total_size = 0

        print(f"{'Archivo':<40} {'Tamaño':<15} {'Última modificación'}")
        print("-" * 70)

        for obj in response['Contents']:
            if obj['Key'].endswith('.RAW'):
                filename = os.path.basename(obj['Key'])
                size_mb = obj['Size'] / (1024 * 1024)
                last_modified = obj['LastModified'].strftime('%Y-%m-%d %H:%M:%S')

                print(f"{filename:<40} {size_mb:>8.2f} MB     {last_modified}")

                archivos.append({
                    'key': obj['Key'],
                    'filename': filename,
                    'size_mb': size_mb,
                    'last_modified': obj['LastModified']
                })

                total_size += size_mb

        print("-" * 70)
        print(f"Total de archivos mostrados: {len(archivos)}")
        print(f"Tamaño total: {total_size:.2f} MB")

        if response.get('IsTruncated', False):
            print(f"\n⚠️  Hay más archivos disponibles (mostrando solo los primeros 20)")

        print()

        return archivos

    except Exception as e:
        print(f"❌ Error explorando archivos: {e}")
        return []


def descargar_archivo_ejemplo(s3_key, carpeta_destino='./datos_radar/'):
    """
    Descarga un archivo de ejemplo del radar

    Args:
        s3_key: ruta completa del archivo en S3
        carpeta_destino: carpeta local donde guardar

    Returns:
        ruta del archivo descargado o None si falla
    """
    os.makedirs(carpeta_destino, exist_ok=True)

    filename = os.path.basename(s3_key)
    local_path = os.path.join(carpeta_destino, filename)

    print("=" * 70)
    print("📥 DESCARGANDO ARCHIVO DE RADAR")
    print("=" * 70)
    print()
    print(f"Archivo: {filename}")
    print(f"Destino: {local_path}")
    print()

    try:
        # Obtener información del archivo primero
        response = s3_client.head_object(Bucket=BUCKET_NAME, Key=s3_key)
        size_mb = response['ContentLength'] / (1024 * 1024)

        print(f"Tamaño: {size_mb:.2f} MB")
        print("Descargando...", end=" ")

        s3_client.download_file(BUCKET_NAME, s3_key, local_path)

        print("✅ COMPLETADO")
        print()
        print(f"Archivo guardado en: {local_path}")
        print()

        return local_path

    except Exception as e:
        print(f"\n❌ Error descargando: {e}")
        return None


def obtener_estadisticas_bucket():
    """Obtiene estadísticas generales del bucket"""
    print("=" * 70)
    print("📊 ESTADÍSTICAS GENERALES DEL BUCKET")
    print("=" * 70)
    print()

    # Buscar últimos 3 días con datos
    print("Buscando días con datos disponibles...")
    print()

    dias_encontrados = []
    for i in range(1, 8):  # Buscar en los últimos 7 días
        fecha = datetime.now() - timedelta(days=i)
        prefix = f"l2_data/{fecha.year}/{fecha.month:02d}/{fecha.day:02d}/"

        try:
            response = s3_client.list_objects_v2(
                Bucket=BUCKET_NAME,
                Prefix=prefix,
                MaxKeys=1
            )

            if response.get('KeyCount', 0) > 0:
                dias_encontrados.append(fecha)
                print(f"✅ {fecha.strftime('%Y-%m-%d')} - Datos disponibles")

            if len(dias_encontrados) >= 3:
                break

        except Exception as e:
            continue

    print()
    print(f"📅 Días con datos encontrados: {len(dias_encontrados)}")

    if dias_encontrados:
        print(f"📅 Día más reciente: {dias_encontrados[0].strftime('%Y-%m-%d')}")

    print()


# ==========================================
# FUNCIÓN PRINCIPAL - DEMO COMPLETO
# ==========================================

def demo_completo():
    """
    Ejecuta una demostración completa del acceso a datos IDEAM
    """
    print("\n")
    print("🛰️" * 35)
    print(" " * 15 + "DEMO: ACCESO A RADAR IDEAM - AWS S3")
    print("🛰️" * 35)
    print("\n")

    # Paso 1: Probar conexión
    if not probar_conexion():
        print("No se pudo conectar al bucket. Verifica tu conexión a internet.")
        return

    input("Presiona ENTER para continuar...")

    # Paso 2: Estadísticas generales
    obtener_estadisticas_bucket()
    input("Presiona ENTER para continuar...")

    # Paso 3: Listar radares disponibles (ayer)
    ayer = datetime.now() - timedelta(days=1)
    radares = listar_radares_disponibles(ayer)

    if not radares:
        print("No se encontraron radares. Intenta con otra fecha.")
        return

    input("Presiona ENTER para continuar...")

    # Paso 4: Explorar archivos de un radar específico
    radar_ejemplo = radares[0]  # Tomar el primer radar disponible
    archivos = explorar_archivos_radar(ayer, radar_ejemplo)

    if not archivos:
        return

    input("Presiona ENTER para continuar...")

    # Paso 5: Preguntar si desea descargar un archivo
    print("=" * 70)
    print("📥 OPCIÓN DE DESCARGA")
    print("=" * 70)
    print()
    print(f"¿Deseas descargar un archivo de ejemplo del radar {radar_ejemplo}?")
    print(f"Tamaño aproximado: {archivos[0]['size_mb']:.2f} MB")
    print()

    respuesta = input("Escribe 'si' para descargar o 'no' para saltar: ").lower().strip()

    if respuesta == 'si':
        archivo_path = descargar_archivo_ejemplo(archivos[0]['key'])

        if archivo_path:
            print("=" * 70)
            print("✅ DESCARGA COMPLETADA")
            print("=" * 70)
            print()
            print("Para procesar este archivo necesitas librerías especializadas:")
            print("  - pip install xradar")
            print("  - pip install pyart")
            print()
            print("Ejemplo de código para procesar:")
            print()
            print("  import xradar as xd")
            print(f"  radar = xd.io.open_iris_datatree('{archivo_path}')")
            print("  print(radar)")
            print()

    print()
    print("=" * 70)
    print("✅ DEMO COMPLETADA")
    print("=" * 70)
    print()
    print("📚 Recursos útiles:")
    print("  - Documentación IDEAM: http://www.pronosticosyalertas.gov.co/archivos-radar")
    print("  - Tutorial Xradar: https://docs.openradarscience.org/projects/xradar/")
    print("  - Registry AWS: https://registry.opendata.aws/ideam-radares/")
    print()


# ==========================================
# MENÚ INTERACTIVO
# ==========================================

def menu_interactivo():
    """Menú interactivo para explorar datos"""

    while True:
        print("\n")
        print("=" * 70)
        print("🛰️  MENÚ - EXPLORADOR DE RADAR IDEAM")
        print("=" * 70)
        print()
        print("1. Probar conexión al bucket")
        print("2. Ver estadísticas generales")
        print("3. Listar radares disponibles")
        print("4. Explorar archivos de un radar específico")
        print("5. Descargar archivo de ejemplo")
        print("6. Demo completo automático")
        print("7. Salir")
        print()

        opcion = input("Selecciona una opción (1-7): ").strip()

        if opcion == '1':
            probar_conexion()
            input("\nPresiona ENTER para volver al menú...")

        elif opcion == '2':
            obtener_estadisticas_bucket()
            input("\nPresiona ENTER para volver al menú...")

        elif opcion == '3':
            fecha_str = input("Fecha (YYYY-MM-DD) o ENTER para ayer: ").strip()
            if fecha_str:
                try:
                    fecha = datetime.strptime(fecha_str, '%Y-%m-%d')
                except:
                    print("Formato de fecha inválido")
                    continue
            else:
                fecha = datetime.now() - timedelta(days=1)

            listar_radares_disponibles(fecha)
            input("\nPresiona ENTER para volver al menú...")

        elif opcion == '4':
            radar = input("Nombre del radar (ej: Guaviare): ").strip()
            fecha_str = input("Fecha (YYYY-MM-DD) o ENTER para ayer: ").strip()

            if fecha_str:
                try:
                    fecha = datetime.strptime(fecha_str, '%Y-%m-%d')
                except:
                    print("Formato de fecha inválido")
                    continue
            else:
                fecha = datetime.now() - timedelta(days=1)

            explorar_archivos_radar(fecha, radar)
            input("\nPresiona ENTER para volver al menú...")

        elif opcion == '5':
            s3_key = input("Ruta completa del archivo en S3: ").strip()
            descargar_archivo_ejemplo(s3_key)
            input("\nPresiona ENTER para volver al menú...")

        elif opcion == '6':
            demo_completo()
            input("\nPresiona ENTER para volver al menú...")

        elif opcion == '7':
            print("\n👋 ¡Hasta luego!\n")
            break

        else:
            print("\n❌ Opción inválida")


# ==========================================
# EJECUCIÓN
# ==========================================

if __name__ == "__main__":
    print("\n¿Qué deseas hacer?")
    print("1. Ejecutar demo automático completo")
    print("2. Usar menú interactivo")
    print()

    modo = input("Selecciona (1 o 2): ").strip()

    if modo == '1':
        demo_completo()
    elif modo == '2':
        menu_interactivo()
    else:
        print("\nOpción inválida. Ejecutando demo automático...\n")
        demo_completo()


¿Qué deseas hacer?
1. Ejecutar demo automático completo
2. Usar menú interactivo



🛰️  MENÚ - EXPLORADOR DE RADAR IDEAM

1. Probar conexión al bucket
2. Ver estadísticas generales
3. Listar radares disponibles
4. Explorar archivos de un radar específico
5. Descargar archivo de ejemplo
6. Demo completo automático
7. Salir

📥 DESCARGANDO ARCHIVO DE RADAR

Archivo: 
Destino: ./datos_radar/


❌ Error descargando: Parameter validation failed:
Invalid length for parameter Key, value: 0, valid min length: 1


🛰️  MENÚ - EXPLORADOR DE RADAR IDEAM

1. Probar conexión al bucket
2. Ver estadísticas generales
3. Listar radares disponibles
4. Explorar archivos de un radar específico
5. Descargar archivo de ejemplo
6. Demo completo automático
7. Salir


❌ Opción inválida


🛰️  MENÚ - EXPLORADOR DE RADAR IDEAM

1. Probar conexión al bucket
2. Ver estadísticas generales
3. Listar radares disponibles
4. Explorar archivos de un radar específico
5. Descargar archivo de ejemplo
6. Demo completo automáti